## create_new_samples_MK_Aug21_NpixelsLoc
Mikhail only submitted one sample in the new request form, but he has 6 samples in this request. 
So I need to populate the db and directories on LightSheetData for the remaining 5 samples

In [2]:
import pickle
import os.path, sys
from datetime import datetime
import pandas as pd
import numpy as np
import datajoint as dj

## Connect to the db

In [3]:
dj.config['database.host'] = 'datajoint00.pni.princeton.edu'
dj.config['database.user'] = os.environ.get('DJ_DB_USER')
dj.config['database.password'] = os.environ.get('DJ_DB_PASS')
dj.conn()

Connecting ahoag@datajoint00.pni.princeton.edu:3306


DataJoint connection (connected) ahoag@datajoint00.pni.princeton.edu:3306

In [4]:
# set up object for light sheet schema
db_lightsheet = dj.create_virtual_module('u19lightserv_lightsheet','u19lightserv_lightsheet')

## Setup for the ingestion

Changes I need to make in the DB (IN THIS ORDER!):
- add 5 entries to Sample() table
- add 5 entries to ClearingBatchSample() table
- add 5 entries to ImagingRequest() table
- add 5 entries to ImagingBatchSample() table
- add 5 entries to ImagingResolutionRequest() table
- add 5x2 entries to ImagingChannel() table, 488 and 555 channels for each sample
- add 5 entries to ProcessingRequest() table
- add 5 entries to ProcessingResolutionRequest() table
- NO entry needed for ProcessingChannel()
- update number_in_batch in ClearingBatch() to 6
- update number_in_imaging_batch to 6 in ImagingBatch()

Changes I need to make on LSD:
- Create the following directory tree for all 5 samples: 
```
MK_Aug21_NpixelsLoc-00X
    └── imaging_request_1
        ├── output
        ├── rawdata
        └── viz
```
And make sure permissions are identical

In [17]:
# Figure out what the sample names need to be
request_name = "MK_Aug21_NpixelsLoc"
db_lightsheet.Request.Sample() & {'request_name':request_name}

username user in the lab,request_name,sample_name,subject_fullname
mkislin,MK_Aug21_NpixelsLoc,MK_Aug21_NpixelsLoc-001,
mkislin,MK_Aug21_NpixelsLoc,MK_Aug21_NpixelsLoc-002,
mkislin,MK_Aug21_NpixelsLoc,MK_Aug21_NpixelsLoc-003,
mkislin,MK_Aug21_NpixelsLoc,MK_Aug21_NpixelsLoc-004,
mkislin,MK_Aug21_NpixelsLoc,MK_Aug21_NpixelsLoc-005,
mkislin,MK_Aug21_NpixelsLoc,MK_Aug21_NpixelsLoc-006,


In [18]:
# Figure out what the clearing batch is
request_name = "MK_Aug21_NpixelsLoc"
db_lightsheet.Request.ClearingBatch() & {'request_name':request_name}

username user in the lab,request_name,clearing_batch_number,clearing_protocol,antibody1,antibody2,antibody1_lot,antibody2_lot,clearing_progress,number_in_batch,perfusion_date,expected_handoff_date,clearer user in the lab,notes_for_clearer,link_to_clearing_spreadsheet
mkislin,MK_Aug21_NpixelsLoc,1,iDISCO abbreviated clearing,,,None,None,complete,6,2021-08-19,2021-08-20,ll3,,None


In [19]:
db_lightsheet.Request.ClearingBatchSample() & {'request_name':request_name}

username user in the lab,request_name,sample_name,clearing_batch_number,clearing_protocol,antibody1,antibody2
mkislin,MK_Aug21_NpixelsLoc,MK_Aug21_NpixelsLoc-001,1,iDISCO abbreviated clearing,,
mkislin,MK_Aug21_NpixelsLoc,MK_Aug21_NpixelsLoc-002,1,iDISCO abbreviated clearing,,
mkislin,MK_Aug21_NpixelsLoc,MK_Aug21_NpixelsLoc-003,1,iDISCO abbreviated clearing,,
mkislin,MK_Aug21_NpixelsLoc,MK_Aug21_NpixelsLoc-004,1,iDISCO abbreviated clearing,,
mkislin,MK_Aug21_NpixelsLoc,MK_Aug21_NpixelsLoc-005,1,iDISCO abbreviated clearing,,
mkislin,MK_Aug21_NpixelsLoc,MK_Aug21_NpixelsLoc-006,1,iDISCO abbreviated clearing,,


In [20]:
db_lightsheet.Request.ImagingRequest() & {'request_name':request_name}

username user in the lab,request_name,sample_name,imaging_request_number,imager user in the lab,imaging_request_date_submitted date that the user submitted the request for imaging,imaging_request_time_submitted time that the user submitted the request for imaging,imaging_performed_date date that the imaging form was submitted by the imager,imaging_progress
mkislin,MK_Aug21_NpixelsLoc,MK_Aug21_NpixelsLoc-001,1,None,2021-08-19,18:06:27,None,incomplete
mkislin,MK_Aug21_NpixelsLoc,MK_Aug21_NpixelsLoc-002,1,None,2021-08-19,18:06:27,None,incomplete
mkislin,MK_Aug21_NpixelsLoc,MK_Aug21_NpixelsLoc-003,1,None,2021-08-19,18:06:27,None,incomplete
mkislin,MK_Aug21_NpixelsLoc,MK_Aug21_NpixelsLoc-004,1,None,2021-08-19,18:06:27,None,incomplete
mkislin,MK_Aug21_NpixelsLoc,MK_Aug21_NpixelsLoc-005,1,None,2021-08-19,18:06:27,None,incomplete
mkislin,MK_Aug21_NpixelsLoc,MK_Aug21_NpixelsLoc-006,1,None,2021-08-19,18:06:27,None,incomplete


In [21]:
db_lightsheet.Request.ImagingBatch() & {'request_name':request_name}

username user in the lab,request_name,clearing_batch_number,imaging_batch_number,imaging_request_number,imager user in the lab,number_in_imaging_batch date that the imaging form was submitted by the imager,imaging_request_date_submitted date that the user submitted the request for imaging,imaging_request_time_submitted time that the user submitted the request for imaging,imaging_performed_date date that the imaging form was submitted by the imager,imaging_progress,imaging_dict
mkislin,MK_Aug21_NpixelsLoc,1,1,1,sj0470,6,2021-08-19,18:06:27,None,in progress,=BLOB=


In [22]:
db_lightsheet.Request.ImagingBatchSample() & {'request_name':request_name}

username user in the lab,request_name,sample_name,clearing_batch_number,imaging_batch_number,imaging_request_number
mkislin,MK_Aug21_NpixelsLoc,MK_Aug21_NpixelsLoc-001,1,1,1
mkislin,MK_Aug21_NpixelsLoc,MK_Aug21_NpixelsLoc-002,1,1,1
mkislin,MK_Aug21_NpixelsLoc,MK_Aug21_NpixelsLoc-003,1,1,1
mkislin,MK_Aug21_NpixelsLoc,MK_Aug21_NpixelsLoc-004,1,1,1
mkislin,MK_Aug21_NpixelsLoc,MK_Aug21_NpixelsLoc-005,1,1,1
mkislin,MK_Aug21_NpixelsLoc,MK_Aug21_NpixelsLoc-006,1,1,1


In [23]:
db_lightsheet.Request.ImagingResolutionRequest() & {'request_name':request_name}

username user in the lab,request_name,sample_name,imaging_request_number,image_resolution,microscope,notes_for_imager,notes_from_imaging
mkislin,MK_Aug21_NpixelsLoc,MK_Aug21_NpixelsLoc-001,1,1.1x,LaVision,,
mkislin,MK_Aug21_NpixelsLoc,MK_Aug21_NpixelsLoc-002,1,1.1x,LaVision,,
mkislin,MK_Aug21_NpixelsLoc,MK_Aug21_NpixelsLoc-003,1,1.1x,LaVision,,
mkislin,MK_Aug21_NpixelsLoc,MK_Aug21_NpixelsLoc-004,1,1.1x,LaVision,,
mkislin,MK_Aug21_NpixelsLoc,MK_Aug21_NpixelsLoc-005,1,1.1x,LaVision,,
mkislin,MK_Aug21_NpixelsLoc,MK_Aug21_NpixelsLoc-006,1,1.1x,LaVision,,


In [24]:
db_lightsheet.Request.ImagingChannel() & {'request_name':request_name}

username user in the lab,request_name,sample_name,imaging_request_number,image_resolution,channel_name,ventral_up whether brain was flipped upside down to be imaged,imaging_date,zoom_body_magnification only applicable for 2x,left_lightsheet_used,right_lightsheet_used,registration,injection_detection,probe_detection,cell_detection,generic_imaging,pixel_type,image_orientation how the imager imaged the sample. Most of the time will be horizontal,numerical_aperture it is not always recorded in metadata so those times it will be NULL,tiling_scheme,tiling_overlap,z_step distance between z planes in microns,number_of_z_planes,rawdata_subfolder,"imspector_channel_index refers to multi-channel imaging - 0 if first (or only) channel in rawdata_subfolder, 1 if second, 2 if third, ...",left_lightsheet_precomputed_spock_jobid,left_lightsheet_precomputed_spock_job_progress,right_lightsheet_precomputed_spock_jobid,right_lightsheet_precomputed_spock_job_progress
mkislin,MK_Aug21_NpixelsLoc,MK_Aug21_NpixelsLoc-001,1,1.1x,488,0,None,nan,1,1,1,0,0,0,0,None,sagittal,nan,1x1,0.0,10.0,None,None,None,None,None,None,None
mkislin,MK_Aug21_NpixelsLoc,MK_Aug21_NpixelsLoc-001,1,1.1x,555,0,None,nan,1,1,0,1,0,0,0,None,sagittal,nan,1x1,0.0,10.0,None,None,None,None,None,None,None
mkislin,MK_Aug21_NpixelsLoc,MK_Aug21_NpixelsLoc-002,1,1.1x,488,0,None,nan,1,1,1,0,0,0,0,None,sagittal,nan,1x1,0.0,10.0,None,None,None,None,None,None,None
mkislin,MK_Aug21_NpixelsLoc,MK_Aug21_NpixelsLoc-002,1,1.1x,555,0,None,nan,1,1,0,1,0,0,0,None,sagittal,nan,1x1,0.0,10.0,None,None,None,None,None,None,None
mkislin,MK_Aug21_NpixelsLoc,MK_Aug21_NpixelsLoc-003,1,1.1x,488,0,None,nan,1,1,1,0,0,0,0,None,sagittal,nan,1x1,0.0,10.0,None,None,None,None,None,None,None
mkislin,MK_Aug21_NpixelsLoc,MK_Aug21_NpixelsLoc-003,1,1.1x,555,0,None,nan,1,1,0,1,0,0,0,None,sagittal,nan,1x1,0.0,10.0,None,None,None,None,None,None,None
mkislin,MK_Aug21_NpixelsLoc,MK_Aug21_NpixelsLoc-004,1,1.1x,488,0,None,nan,1,1,1,0,0,0,0,None,sagittal,nan,1x1,0.0,10.0,None,None,None,None,None,None,None
mkislin,MK_Aug21_NpixelsLoc,MK_Aug21_NpixelsLoc-004,1,1.1x,555,0,None,nan,1,1,0,1,0,0,0,None,sagittal,nan,1x1,0.0,10.0,None,None,None,None,None,None,None
mkislin,MK_Aug21_NpixelsLoc,MK_Aug21_NpixelsLoc-005,1,1.1x,488,0,None,nan,1,1,1,0,0,0,0,None,sagittal,nan,1x1,0.0,10.0,None,None,None,None,None,None,None
mkislin,MK_Aug21_NpixelsLoc,MK_Aug21_NpixelsLoc-005,1,1.1x,555,0,None,nan,1,1,0,1,0,0,0,None,sagittal,nan,1x1,0.0,10.0,None,None,None,None,None,None,None


In [25]:
db_lightsheet.Request.ProcessingRequest() & {'request_name':request_name}

username user in the lab,request_name,sample_name,imaging_request_number,processing_request_number,processor user in the lab,processing_request_date_submitted date that the user submitted the request for processing,processing_request_time_submitted time that the user submitted the request for processing,processing_performed_date date that the processing form was submitted by the processor,processing_progress
mkislin,MK_Aug21_NpixelsLoc,MK_Aug21_NpixelsLoc-001,1,1,mkislin,2021-08-19,18:06:27,None,incomplete
mkislin,MK_Aug21_NpixelsLoc,MK_Aug21_NpixelsLoc-002,1,1,mkislin,2021-08-19,18:06:27,None,incomplete
mkislin,MK_Aug21_NpixelsLoc,MK_Aug21_NpixelsLoc-003,1,1,mkislin,2021-08-19,18:06:27,None,incomplete
mkislin,MK_Aug21_NpixelsLoc,MK_Aug21_NpixelsLoc-004,1,1,mkislin,2021-08-19,18:06:27,None,incomplete
mkislin,MK_Aug21_NpixelsLoc,MK_Aug21_NpixelsLoc-005,1,1,mkislin,2021-08-19,18:06:27,None,incomplete
mkislin,MK_Aug21_NpixelsLoc,MK_Aug21_NpixelsLoc-006,1,1,mkislin,2021-08-19,18:06:27,None,incomplete


In [26]:
db_lightsheet.Request.ProcessingResolutionRequest() & {'request_name':request_name}

username user in the lab,request_name,sample_name,imaging_request_number,processing_request_number,image_resolution,ventral_up whether brain was flipped upside down to be imaged,atlas_name,final_orientation,notes_for_processor,notes_from_processing,lightsheet_pipeline_spock_jobid the jobid from the final step in the light sheet processing pipeline,lightsheet_pipeline_spock_job_progress the spock job status code for the final step in the light sheet processing pipeline,brainpipe_commit the commit that is checked out on the machine at the time the job was submitted
mkislin,MK_Aug21_NpixelsLoc,MK_Aug21_NpixelsLoc-001,1,1,1.1x,0,princeton_mouse_atlas,sagittal,,,None,None,None
mkislin,MK_Aug21_NpixelsLoc,MK_Aug21_NpixelsLoc-002,1,1,1.1x,0,princeton_mouse_atlas,sagittal,,,None,None,None
mkislin,MK_Aug21_NpixelsLoc,MK_Aug21_NpixelsLoc-003,1,1,1.1x,0,princeton_mouse_atlas,sagittal,,,None,None,None
mkislin,MK_Aug21_NpixelsLoc,MK_Aug21_NpixelsLoc-004,1,1,1.1x,0,princeton_mouse_atlas,sagittal,,,None,None,None
mkislin,MK_Aug21_NpixelsLoc,MK_Aug21_NpixelsLoc-005,1,1,1.1x,0,princeton_mouse_atlas,sagittal,,,None,None,None
mkislin,MK_Aug21_NpixelsLoc,MK_Aug21_NpixelsLoc-006,1,1,1.1x,0,princeton_mouse_atlas,sagittal,,,None,None,None


In [27]:
db_lightsheet.Request.ProcessingChannel() & {'request_name':request_name}

username user in the lab,request_name,sample_name,imaging_request_number,image_resolution,channel_name,ventral_up whether brain was flipped upside down to be imaged,processing_request_number,lightsheet_channel_str,imspector_version,datetime_processing_started,datetime_processing_completed,intensity_correction,metadata_xml_string The entire metadata xml string. Sometimes it is not available so those times it will be NULL,left_lightsheet_stitched_precomputed_spock_jobid,left_lightsheet_stitched_precomputed_spock_job_progress,right_lightsheet_stitched_precomputed_spock_jobid,right_lightsheet_stitched_precomputed_spock_job_progress,blended_precomputed_spock_jobid,blended_precomputed_spock_job_progress,downsized_precomputed_spock_jobid,downsized_precomputed_spock_job_progress,registered_precomputed_spock_jobid,registered_precomputed_spock_job_progress


## Perform the inserts/updates

In [16]:
sample_names = ['MK_Aug21_NpixelsLoc-'+ str(ii).zfill(3) for ii in range(2,7)]
existing_sample_dict = (db_lightsheet.Request.Sample() & {'request_name':request_name}).fetch1()
existing_clearing_batch_dict = (
    db_lightsheet.Request.ClearingBatch() & {'request_name':request_name}).fetch1()
existing_clearing_batch_sample_dict = (
    db_lightsheet.Request.ClearingBatchSample() & {'request_name':request_name}).fetch1()
existing_imaging_batch_dict = (
    db_lightsheet.Request.ImagingBatch() & {'request_name':request_name}).fetch1()
existing_imaging_request_dict = (
    db_lightsheet.Request.ImagingRequest() & {'request_name':request_name}).fetch1()
existing_imaging_batch_sample_dict = (
    db_lightsheet.Request.ImagingBatchSample() & {'request_name':request_name}).fetch1()
existing_imaging_resolution_request_dict = (
    db_lightsheet.Request.ImagingResolutionRequest() & {'request_name':request_name}).fetch1()
existing_imaging_channel_dict_list = (
    db_lightsheet.Request.ImagingChannel() & {'request_name':request_name}).fetch(as_dict=True)
existing_processing_request_dict = (
    db_lightsheet.Request.ProcessingRequest() & {'request_name':request_name}).fetch1()
existing_processing_resolution_request_dict = (
    db_lightsheet.Request.ProcessingResolutionRequest() & {'request_name':request_name}).fetch1() 

connection = db_lightsheet.Request.connection
with connection.transaction:
    for sample_name in sample_names:
        # Sample()
        sample_insert_dict = existing_sample_dict.copy()
        sample_insert_dict['sample_name'] = sample_name
        # ClearingBatch()
        clearing_batch_update_dict = existing_clearing_batch_dict.copy()
        clearing_batch_update_dict['number_in_batch'] = 6
        # ClearingBatchSample()
        clearing_batch_sample_insert_dict = existing_clearing_batch_sample_dict.copy()
        clearing_batch_sample_insert_dict['sample_name'] = sample_name
        # ImagingRequest()
        imaging_request_insert_dict = existing_imaging_request_dict.copy()
        imaging_request_insert_dict['sample_name'] = sample_name
        # ImagingBatch()
        imaging_batch_update_dict = existing_imaging_batch_dict.copy()
        imaging_batch_update_dict['number_in_imaging_batch'] = 6
        # ImagingBatchSample()
        imaging_batch_sample_insert_dict = existing_imaging_batch_sample_dict.copy()
        imaging_batch_sample_insert_dict['sample_name'] = sample_name
        # ImagingResolutionRequest()
        imaging_resolution_request_insert_dict = existing_imaging_resolution_request_dict.copy()
        imaging_resolution_request_insert_dict['sample_name'] = sample_name
        # ImagingChannel()
        imaging_channel_insert_dict_list = []
        for existing_imaging_channel_dict in existing_imaging_channel_dict_list:  
            imaging_channel_insert_dict = existing_imaging_channel_dict.copy()
            imaging_channel_insert_dict['sample_name'] = sample_name
            imaging_channel_insert_dict_list.append(imaging_channel_insert_dict)
        # ProcessingRequest()
        processing_request_insert_dict = existing_processing_request_dict.copy()
        processing_request_insert_dict['sample_name'] = sample_name
        # ProcessingResolutionRequest()
        processing_resolution_request_insert_dict = existing_processing_resolution_request_dict.copy()
        processing_resolution_request_insert_dict['sample_name'] = sample_name
        
        db_lightsheet.Request.Sample().insert1(
            sample_insert_dict,skip_duplicates=True)
        
        db_lightsheet.Request.ClearingBatchSample().insert1(
            clearing_batch_sample_insert_dict,skip_duplicates=True)
        
        db_lightsheet.Request.ImagingRequest().insert1(
            imaging_request_insert_dict,skip_duplicates=True)
        
        db_lightsheet.Request.ImagingBatchSample().insert1(
            imaging_batch_sample_insert_dict,skip_duplicates=True)
        
        db_lightsheet.Request.ImagingResolutionRequest().insert1(
            imaging_resolution_request_insert_dict,skip_duplicates=True)
        
        db_lightsheet.Request.ImagingChannel().insert(
            imaging_channel_insert_dict_list,skip_duplicates=True)
        
        db_lightsheet.Request.ProcessingRequest.insert1(
            processing_request_insert_dict,skip_duplicates=True)
        
        db_lightsheet.Request.ProcessingResolutionRequest.insert1(
            processing_resolution_request_insert_dict,skip_duplicates=True)
        
    # Now outside of loop do the update that we only need to do once to the batch tables
    db_lightsheet.Request.ClearingBatch().update1(
            clearing_batch_update_dict)
    db_lightsheet.Request.ImagingBatch().update1(
            imaging_batch_update_dict)
        